In [3]:
import pandas as pd

train_df = pd.read_csv('datasets/digit/train.csv')
predict_df = pd.read_csv('datasets/digit/test.csv')

In [21]:
from sklearn.model_selection import train_test_split
import numpy as np

train_labels = train_df['label']
# train_images = train_df.drop('label',axis=1).values.reshape((train_df.shape[0], 28, 28))
train_images = train_df.drop('label',axis=1).values.astype(np.float)
train_images = np.multiply(train_images, 1.0 / 255.0)

X_train, X_test, y_train, y_test = train_test_split(train_images, train_labels, random_state=42)


predict_images = predict_df.values.astype(np.float)
predict_images = np.multiply(predict_images, 1.0 / 255.0)

In [23]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=10000).fit(X_train, y_train)

print("Training set score : {:.3f}".format(lr.score(X_train, y_train)))
print("Test set score : {:.3f}".format(lr.score(X_test, y_test)))

predictions = lr.predict(predict_images)

Training set score : 0.946
Test set score : 0.920


In [31]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(solver='lbfgs', max_iter=1000, activation='tanh', alpha=5,
                    hidden_layer_sizes=[100, 100], random_state=42)
mlp.fit(X_train, y_train)

print("Training set score : {:.3f}".format(mlp.score(X_train, y_train)))
print("Test set score : {:.3f}".format(mlp.score(X_test, y_test)))

predictions = mlp.predict(predict_images)

Training set score : 1.000
Test set score : 0.977


In [32]:
out = { 'ImageId': [ i for i in range(1, len(predictions) + 1)], 'Label': predictions }
out_df = pd.DataFrame(out)
out_df.to_csv('/tmp/submission-digit.csv', encoding='utf-8', index=False)